In [47]:
import cv2 as cv
import numpy as np
from skimage.filters import gabor_kernel

In [48]:
#Thresholding values for powerline color in HSV color space
low_H = 0
low_S = 45
low_V = 0

high_H = 90
high_S = 255
high_V = 255

In [49]:
#Get angle between two vectors with arcat
def angle(v1, v2, acute):
    if v1[0]-v2[0] == 0:
        return np.pi/2
    angle = np.arctan((v1[1]-v2[1])/(v1[0]-v2[0]))
    return np.rad2deg(angle)

In [51]:
def draw_lines(lines, mask):
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv.line(mask, (x1, y1), (x2, y2), (255, 255, 255), 3)
    return mask

In [52]:
def average_angle(lines):
    avrg = 0.0
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                avrg += angle((x1,y1), (x2,y2), True)
        avrg = avrg / lines.shape[0]
    return avrg

In [53]:
img = cv.imread("6.JPG")#read the image

In [54]:
#Convert image to HSV color space and threshold
img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

thresh_hsv = cv.inRange(img_hsv, (low_H, low_S, low_V), (high_H, high_S, high_V))
thresh_hsv = 255 - thresh_hsv

cv.imwrite('results/1_thresh_HSV.jpg',thresh_hsv)

True

In [61]:
#Apply 
thresh = np.zeros_like(thresh_hsv)#store threshold

THETA_A = -20
THETA_B = 40

kernel = np.ones((1,1),np.uint8)#kernel for dilation

#Apply Gabor filter for theta with predefined filter arguments
for theta in range(THETA_A, THETA_B, 2):
    kern = np.real(gabor_kernel(1 / 12, theta=np.deg2rad(theta), sigma_x=35 / 6.3, sigma_y=35 / 6.3))
    fimg = cv.filter2D(img, cv.CV_8UC3, kern)
    accum = np.zeros_like(fimg)
    np.maximum(accum, fimg, accum)
    accum = cv.cvtColor(accum,cv.COLOR_BGR2GRAY)
    ret, thresh_tmp = cv.threshold(accum, 9, 255, cv.THRESH_BINARY) #threshold from gabor filter

    thresh_tmp = cv.dilate(thresh_tmp, kernel, iterations=3)

    thresh = cv.bitwise_or(thresh, thresh_tmp) # collecting overal lines

thresh = cv.bitwise_and(thresh_hsv,thresh) # hsv and gabor threshold bitwise and
thresh = cv.dilate(thresh, kernel, iterations=1)

cv.imwrite('results/2_thres_GABOR.png', thresh)#save gabor threshold after bitwise and with HSV threshold

True

In [62]:
#Find line segments with probabilistic Hough Transform
lines = cv.HoughLinesP(thresh, rho=0.2, theta=np.pi / 90, threshold=40, minLineLength=80, maxLineGap=3)
line_mask = np.zeros_like(thresh)#binary mask image of drawn line segments
line_mask = draw_lines(lines,line_mask)
cv.imwrite('results/3_mask_hough.png', line_mask)

True

In [63]:
angles = np.array([])#angles of line segments

if lines is not None:
    for line in lines:
        for x1, y1, x2, y2 in line:
            if angles.shape[0] == 0:
                angles = np.array([angle([x1,y1], [x2,y2], True)])
            else:
                angles = np.append(angles,[angle([x1,y1], [x2,y2], True)],axis=0)

In [64]:
ANGLE_THRESHOLD = 7
avrg_angle = average_angle(lines)

angle_label = np.zeros((angles.shape[0],1)).astype(int)
i = 0
for a in angles:
    if a > avrg_angle + ANGLE_THRESHOLD or a < avrg_angle - ANGLE_THRESHOLD:
        angle_label[i] = 1
    i += 1 
#Remove outlier segments
i = 0
for p in angle_label:
    if p == 1:
        lines = np.delete(lines, i, axis=0)
        i -= 1
    i+=1
    
#Make mask after removing outliers
line_mask = np.zeros_like(line_mask)
line_mask = draw_lines(lines, line_mask)
cv.imwrite('results/4_mask_filtered.png', line_mask)

True

In [65]:
#Find line segments with probabilistic Hough Transform
lines_over = cv.HoughLinesP(line_mask, rho=7, theta=np.pi / 90, threshold=200, minLineLength=10, maxLineGap=500)
line_mask_over = np.zeros_like(thresh)#binary mask image of drawn line segments
line_mask_over = draw_lines(lines_over,line_mask_over)
cv.imwrite('results/5_mask_overline.png', line_mask_over)

True

In [68]:
#label found lines and remove small ones

ret, labels = cv.connectedComponents(line_mask_over)
assert ret != 0,"NO lines"
# Map component labels to hue val
label_hue = np.uint8(179*labels/np.max(labels))
blank_ch = 255*np.ones_like(label_hue)
labeled_img = cv.merge([label_hue, blank_ch, blank_ch])
# cvt to BGR for display
labeled_img = cv.cvtColor(labeled_img, cv.COLOR_HSV2BGR)

# set bg label to black
labeled_img[label_hue==0] = 0

cv.imwrite('results/6_labeled.png',labeled_img)

avrg_surface = 0
for i in range(1,ret):
    avrg_surface += labels[labels==i].shape[0]
avrg_surface /= ret - 1

SURFACE_THRESH = 0.5

for i in range(1,ret):
    if labels[labels==i].shape[0] < avrg_surface*SURFACE_THRESH:
        labeled_img[labels==i] = 0
cv.imwrite('results/7_labeled_filtered.png',labeled_img)

labeled_img = cv.cvtColor(labeled_img, cv.COLOR_BGR2GRAY)

ret, result = cv.threshold(labeled_img, 1, 255, cv.THRESH_BINARY)

cv.imwrite('results/8_result.png',result)

True

In [70]:
#Inpaint: remove detected lines from the image
dst = cv.inpaint(img,result,3,cv.INPAINT_TELEA)
cv.imwrite('results/9_inpaint.png', dst)

True